In [9]:
from extract_code import Extract
base_dir = r"C:\Users\FASI OWAIZ AHMED\Desktop\Code_gen\Data"
res=Extract.extract_all_functions(base_dir)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import chromadb
from chromadb.config import Settings

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings[0].numpy()


#chromaDB
client = chromadb.PersistentClient(path="./code_index")  #NEW
collection = client.get_or_create_collection("code_snippets")


#ast of all languages
from extract_code import Extract
base_dir = r"C:\Users\FASI OWAIZ AHMED\Desktop\Code_gen\Data"
extracted=Extract.extract_all_functions(base_dir)

# vectorizing and storing in chromaDB
for idx, item in enumerate(extracted):
    text_for_embedding = item["function_name"] + " "+ item["code"] + " " + item["docstring"]
    vector = get_embedding(text_for_embedding)

    collection.add(
        documents=[text_for_embedding],
        embeddings=[vector],
        metadatas=[{
            "function_name": item["function_name"],
            "file_path": item["file_path"],
            "language": item["language"],
            "docstring": item["docstring"]
        }],
        ids=[str(idx)]
    )


c:\Users\FASI OWAIZ AHMED\Desktop\Code_gen\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\FASI OWAIZ AHMED\Desktop\Code_gen\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [4]:
query_vector = get_embedding("find binary search in python")

results = collection.query(
    query_embeddings=[query_vector],  # not query_texts
    n_results=5
)
print(results)

{'ids': [['104', '85', '112', '124', '136']], 'embeddings': None, 'documents': [['linearSearch  ', 'binarySearch  ', 'permute  ', 'sieve  ', 'find  ']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'file_path': 'C:\\Users\\FASI OWAIZ AHMED\\Desktop\\Code_gen\\Data\\Java\\linear search.java', 'function_name': 'linearSearch', 'docstring': '', 'language': 'java'}, {'file_path': 'C:\\Users\\FASI OWAIZ AHMED\\Desktop\\Code_gen\\Data\\Java\\binary search.java', 'language': 'java', 'function_name': 'binarySearch', 'docstring': ''}, {'language': 'java', 'file_path': 'C:\\Users\\FASI OWAIZ AHMED\\Desktop\\Code_gen\\Data\\Java\\permutations.java', 'docstring': '', 'function_name': 'permute'}, {'function_name': 'sieve', 'docstring': '', 'file_path': 'C:\\Users\\FASI OWAIZ AHMED\\Desktop\\Code_gen\\Data\\Java\\sieve of eratosthenes.java', 'language': 'java'}, {'docstring': '', 'function_name': 'find', 'file_path': 'C:\\Users\\FASI OWAIZ AHMED\\De

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import chromadb
from chromadb.config import Settings
import faiss
import numpy as np
import pickle

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings[0].numpy()


#chromaDB
client = chromadb.PersistentClient(path="./code_index")  # ✅ NEW
collection = client.get_or_create_collection("code_snippets")


#ast of all languages
from extract_code import Extract
base_dir = r"C:\Users\FASI OWAIZ AHMED\Desktop\Code_gen\Data"
extracted=Extract.extract_all_functions(base_dir)


# vectorizing and storing in chromaDB
faiss_embeddings = []
faiss_metadata = []

for idx, item in enumerate(extracted):
    text_for_embedding = item["function_name"] + " "+ item["code"] + " " + item["docstring"]
    vector = get_embedding(text_for_embedding)

    collection.add(
        documents=[text_for_embedding],
        embeddings=[vector],
        metadatas=[{
            "function_name": item["function_name"],
            "file_path": item["file_path"],
            "language": item["language"],
            "docstring": item["docstring"]
        }],
        ids=[str(idx)]
    )
    faiss_embeddings.append(vector)
    faiss_metadata.append(item)
    # Cleanup memory
    del vector
    torch.cuda.empty_cache()
embedding_matrix = np.vstack(faiss_embeddings).astype("float32")
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

# Save FAISS index and metadata
faiss.write_index(index, "code_index.faiss")
with open("faiss_metadata.pkl", "wb") as f:
    pickle.dump(faiss_metadata, f)



c:\Users\FASI OWAIZ AHMED\Desktop\Code_gen\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\FASI OWAIZ AHMED\Desktop\Code_gen\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [3]:
# Load everything
import faiss
import pickle
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
import chromadb
from chromadb.config import Settings
import faiss
import numpy as np
import pickle

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings[0].numpy()





index = faiss.read_index("code_index.faiss")
with open("faiss_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

# Embed query
query = "find binary search in python"
query_vector = get_embedding(query).reshape(1, -1).astype("float32")

D, I = index.search(query_vector, 5)

# Show results
for i in I[0]:
    print(metadata[i]["function_name"], "->", metadata[i]["file_path"])


c:\Users\FASI OWAIZ AHMED\Desktop\Code_gen\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\FASI OWAIZ AHMED\Desktop\Code_gen\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


: 